In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from valuation.val import perform_valuation

In [2]:
comp = pd.read_csv(f'{DATA_FOLDER}/comp_1959-2019.csv', index_col=0)
crsp = pd.read_csv(f'{DATA_FOLDER}/crsp_1959-2019.csv', index_col=0)
beta = pd.read_csv(f'{DATA_FOLDER}/capm_beta_weekly_1-3y_1959-2019.csv', index_col=0)

comp['jdate'] = pd.to_datetime(comp['jdate'])
comp.permno = comp.permno.astype(int)

beta['date']=pd.to_datetime(beta['DATE'].astype(str))
beta['jdate']=beta['date']+MonthEnd(0)
beta['permno'] = beta.index
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
FF = pd.read_csv(f'{PROJECT_FOLDER}/lookup_tables/FF3.csv')
FF['date'] = FF['Unnamed: 0']
FF['date'] = pd.to_datetime(FF['date'].astype(str), format='%Y%m%d')
FF['jdate'] = FF['date']+MonthEnd(0)
FF = FF[['jdate', 'date', 'RF']]
FF['RF'] = (1+FF['RF']/100).pow(365)-1

beta = beta.merge(FF[['jdate', 'RF']],  on=['jdate'])
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

In [4]:
comp.sort_values(['permno','year'], ascending = [True,False], inplace=True)
for i in range(11):
    comp[f'ni_{i}'] = comp.groupby('permno')['ni'].shift(i)
    comp[f'ni_{i}'] = comp[f'ni_{i}'].fillna(0)

In [5]:
comp.sort_values(['permno','jdate'], ascending = [True,True], inplace=True)
crsp['logret'] = np.log(1+crsp['retadj'])
crsp['termlogret'] = crsp.groupby('permno')['logret'].transform('sum') - crsp.groupby('permno')['logret'].transform('cumsum') + np.log(1-0.35) 
crsp['termcumret'] = np.exp(crsp['termlogret'])-1
crsp1 = crsp.copy()

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [6]:
crsp[['ticker', 'jdate', 'logret', 'termlogret']].head(3)

,ticker,jdate,logret,termlogret
0,OMFGA,1986-01-31,0.000000,-3.426515
1,OMFGA,1986-02-28,-0.297252,-3.129264
2,OMFGA,1986-03-31,0.311436,-3.440700


In [7]:
for x in ['logret_3', 'logret_6', 'logret_12', 'logret_24', 
      'logret_36', 'logret_48', 'logret_60']:
    crsp1.loc[crsp1[x].isnull(), x] = crsp1['termlogret']


for x in ['cumret_3', 'cumret_6', 'cumret_12', 'cumret_24', 
      'cumret_36', 'cumret_48', 'cumret_60']:
    crsp1.loc[crsp1[x].isnull(), x] = crsp1['termcumret']

In [8]:
valdf = comp.merge(beta, on=['jdate', 'permno'])
valdf = perform_valuation(valdf, n=5, rp=0.07, tg=0.05)
valdf = valdf[['permno', 'gvkey', 'year', 'PV', 'be']]
valdf.year = valdf.year + 1
val_crsp_mdf = valdf.merge(crsp1, on=['year', 'permno'])
val_crsp_mdf['vt'] = val_crsp_mdf['PV']*1000
val_crsp_mdf['be'] = val_crsp_mdf['be']*1000

In [21]:
val_df1 = val_crsp_mdf[['permno', 'ticker',  'exchcd', 'year', 'jdate', 'exchcd',
                        'PV', 'vt','me', 'be', 'prc',
                        'logret_3', 'logret_6', 'logret_24', 'cumret_3', 'cumret_6', 'cumret_24',
                        'logret_12', 'logret_36', 'logret_48', 'logret_60', 
                        'cumret_12', 'cumret_36', 'cumret_48', 'cumret_60',]]
val_df1.sort_values(['permno','jdate'], ascending = [True,False], inplace=True)

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
val_df2 = val_df1.copy()
for m in [3, 6, 12, 24, 36, 48, 60]:
    val_df2[f'vt+{m}'] = val_df1.groupby('permno')['vt'].shift(m)
    val_df2[f'me+{m}'] = val_df1.groupby('permno')['me'].shift(m)
    val_df2[f'be+{m}'] = val_df1.groupby('permno')['be'].shift(m)

In [23]:
val_df3 = val_df2.copy()
for m in [3, 6, 12, 24, 36, 48, 60]:
    val_df3['vt'] = np.log(val_df2.vt)
    val_df3[f'vt+{m}'] = np.log(val_df2[f'vt+{m}'].astype(float))
    val_df3['me'] = np.log(val_df2['me'].astype(float))
    val_df3[f'me+{m}'] = np.log(val_df2[f'me+{m}'].astype(float))
    val_df3[f'r_t+{m}'] = val_df3[f'logret_{m}']
    val_df3[f'r*_t+{m}'] = val_df3[f'vt+{m}'] -  val_df3[f'vt']
    val_df3[f'r_unchg+{m}'] = - (1-0.975) * (val_df3['me'] -  val_df3['vt'])
    val_df3[f'r_chg+{m}'] = 0.975 * ( (val_df3[f'me+{m}'] -  val_df3[f'vt+{m}']) - ((val_df3[f'me'] -  val_df3['vt'])) )
    

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/mmw/Installations/venvs/mlval/lib/python3.7/s

In [24]:
val_df3 = val_df3[(val_df3.year >=1980) & (val_df3.year<=2014)]
val_df3 = val_df3.drop_duplicates(subset=['permno', 'jdate'])

In [25]:
val_df3['misp'] = val_df3['me'] - val_df3['vt']
# val_df3 = val_df3.dropna(subset=['misp', 'r_t+60'])

In [26]:
val_df3[(val_df3.ticker=='AAPL') & (val_df3.ticker=='AAPL')]

,permno,ticker,exchcd,year,jdate,exchcd,PV,vt,me,be,prc,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
227695,14593,AAPL,3,2014,2014-12-31,3,1.770114e+06,21.294310,20.281842,140038000.0,110.379997,1.234664e-01,0.135415,0.086288,1.314120e-01,0.145011,0.090120,-0.030873,0.481271,0.425671,1.062266,-0.030401,0.618130,0.530617,1.892920,21.282744,20.401370,131806000.0,21.282744,20.388632,131806000.0,21.282744,20.184748,131806000.0,19.039989,20.227264,143417000.0,18.827261,20.573469,154268000.0,18.515270,20.430342,165551000.0,17.827487,20.989289,107573000.0,1.234664e-01,-0.011566,0.025312,0.127816,0.135415,-0.011566,0.025312,0.115396,-0.030873,-0.011566,0.025312,-0.083390,0.086288,-2.254321,0.025312,2.144749,0.481271,-2.467049,0.025312,2.689709,0.425671,-2.779040,0.025312,2.854351,1.062266,-3.466822,0.025312,4.069912,-1.012468
227694,14593,AAPL,3,2014,2014-11-30,3,1.770114e+06,21.294310,20.363021,140038000.0,118.930000,8.073458e-02,0.098786,-0.035159,8.408313e-02,0.103830,-0.034548,0.011311,0.422028,0.475154,0.893456,0.011375,0.525051,0.608261,1.443561,21.282744,20.433244,131806000.0,21.282744,20.436529,131806000.0,21.282744,20.307086,131806000.0,19.039989,20.194492,143417000.0,18.827261,20.598078,154268000.0,18.515270,20.557723,165551000.0,17.827487,20.895085,107573000.0,8.073458e-02,-0.011566,0.023282,0.079744,0.098786,-0.011566,0.023282,0.082947,0.011311,-0.011566,0.023282,-0.043260,-0.035159,-2.254321,0.023282,2.033647,0.422028,-2.467049,0.023282,2.634554,0.475154,-2.779040,0.023282,2.899398,0.893456,-3.466822,0.023282,3.898914,-0.931289
227693,14593,AAPL,3,2014,2014-10-31,3,1.770114e+06,21.294310,20.266617,140038000.0,108.000000,8.535344e-02,0.154978,0.087004,8.910193e-02,0.167632,0.090901,0.117366,0.502230,0.774818,0.919231,0.124531,0.652402,1.170196,1.507362,21.282744,20.341167,131806000.0,21.282744,20.396356,131806000.0,21.282744,20.317179,131806000.0,19.039989,20.221451,143417000.0,18.827261,20.581592,154268000.0,18.515270,20.761118,165551000.0,17.827487,20.823389,107573000.0,8.535344e-02,-0.011566,0.025692,0.083963,0.154978,-0.011566,0.025692,0.137772,0.117366,-0.011566,0.025692,0.060574,0.087004,-2.254321,0.025692,2.153926,0.502230,-2.467049,0.025692,2.712473,0.774818,-2.779040,0.025692,3.191702,0.919231,-3.466822,0.025692,3.923004,-1.027693
227692,14593,AAPL,3,2014,2014-09-30,3,1.770114e+06,21.294310,20.197353,140038000.0,100.750000,9.523085e-02,0.218697,0.152168,9.991274e-02,0.244454,0.164356,0.106742,0.479315,0.875258,0.883744,0.112648,0.614968,1.399495,1.419943,21.294310,20.281842,140038000.0,21.282744,20.401370,131806000.0,21.282744,20.259658,131806000.0,19.039989,20.227597,143417000.0,18.827261,20.487607,154268000.0,18.515270,20.794088,165551000.0,17.827487,20.718406,107573000.0,9.523085e-02,0.000000,0.027424,0.082377,0.218697,-0.011566,0.027424,0.210192,0.106742,-0.011566,0.027424,0.072024,0.152168,-2.254321,0.027424,2.227451,0.479315,-2.467049,0.027424,2.688370,0.875258,-2.779040,0.027424,3.291380,0.883744,-3.466822,0.027424,3.888178,-1.096957
227691,14593,AAPL,3,2014,2014-08-31,3,1.770114e+06,21.294310,20.235109,140038000.0,102.500000,1.526164e-01,0.233351,0.071499,1.648781e-01,0.262825,0.074117,0.111579,0.524230,0.866375,0.796101,0.118043,0.689157,1.378275,1.216879,21.294310,20.363021,140038000.0,21.282744,20.433244,131806000.0,21.282744,20.281716,131806000.0,19.039989,20.164149,143417000.0,18.827261,20.557326,154268000.0,18.515270,20.818063,165551000.0,17.827487,20.664931,107573000.0,1.526164e-01,0.000000,0.026480,0.124714,0.233351,-0.011566,0.026480,0.204458,0.111579,-

In [27]:
val_df3 = val_df3.replace([np.inf, -np.inf], np.nan)

In [28]:
val_df3.to_csv(f'{DATA_FOLDER}/decomp_dcf5y_hist_1980_2014.csv')

In [29]:
val_df3.describe()

,permno,exchcd,year,exchcd,PV,vt,me,be,prc,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
count,1.738053e+06,1.738053e+06,1.738053e+06,1.738053e+06,1.735677e+06,1.027597e+06,1.736706e+06,1.566521e+06,1.736706e+06,1.738040e+06,1.738022e+06,1.737925e+06,1.738050e+06,1.738050e+06,1.738050e+06,1.737981e+06,1.737881e+06,1.737845e+06,1.737809e+06,1.738050e+06,1.738050e+06,1.738050e+06,1.738050e+06,1.009472e+06,1.704274e+06,1.537467e+06,991182.000000,1.671904e+06,1.508703e+06,954754.000000,1.609125e+06,1.452634e+06,891432.000000,1.493273e+06,1.348763e+06,834260.000000,1.388045e+06,1.253687e+06,781862.000000,1.292007e+06,1.166143e+06,734274.000000,1.204856e+06,1.086652e+06,1.738040e+06,981972.000000,1.027350e+06,981574.000000,1.738022e+06,937135.000000,1.027350e+06,936767.000000,1.737981e+06,847116.000000,1.027350e+06,846829.000000,1.737925e+06,760042.000000,1.027350e+06,759795.000000,1.737881e+06,694996.000000,1.027350e+06,694792.000000,1.737845e+06,644396.000000,1.027350e+06,644224.000000,1.737809e+06,589613.000000,1.027350e+06,589466.000000,1.027350e+06
mean,5.740075e+04,2.074157e+00,1.998024e+03,2.074157e+00,8.648392e+03,1.241536e+01,1.209479e+01,1.397359e+06,2.887390e+01,7.662530e-04,-3.427422e-04,-1.454529e-02,3.444562e-02,6.780467e-02,2.504100e-01,-5.074119e-03,-1.646874e-02,-2.317023e-02,-2.764767e-02,1.336477e-01,3.718246e-01,4.866846e-01,6.171464e-01,1.245424e+01,1.212892e+01,1.438574e+06,12.495291,1.216415e+01,1.481092e+06,12.581294,1.223328e+01,1.569944e+06,12.715244,1.236505e+01,1.750557e+06,12.833849,1.250074e+01,1.942688e+06,12.933609,1.262803e+01,2.153769e+06,13.007097,1.275340e+01,2.379117e+06,7.662530e-04,-0.001868,-2.842209e-03,0.028882,-3.427422e-04,-0.003731,-2.842209e-03,0.059396,-5.074119e-03,-0.008046,-2.842209e-03,0.124321,-1.454529e-02,-0.039691,-2.842209e-03,0.265367,-1.646874e-02,-0.080986,-2.842209e-03,0.422915,-2.317023e-02,-0.143381,-2.842209e-03,0.589878,-2.764767e-02,-0.085453,-2.842209e-03,0.636752,1.136884e-01
std,2.598535e+04,9.371966e-01,9.710864e+00,9.371966e-01,2.439523e+06,2.840529e+00,2.222516e+00,6.924538e+06,8.091590e+02,2.610698e-01,3.751174e-01,7.670879e-01,2.958667e-01,4.505735e-01,1.279397e+00,5.402186e-01,9.357184e-01,1.072913e+00,1.195204e+00,8.826344e-01,1.663627e+00,2.165338e+00,3.040632e+00,2.846219e+00,2.225717e+00,7.083537e+06,2.851174,2.228153e+00,7.244066e+06,2.860105,2.231412e+00,7.568897e+06,2.834284,2.239422e+00,8.203436e+06,2.807220,2.246521e+00,8.840521e+06,2.779886,2.258431e+00,9.512509e+06,2.760765,2.268974e+00,1.020533e+07,2.610698e-01,0.471945,4.540470e-02,0.512282,3.751174e-01,0.680595,4.540470e-02,0.734469,5.402186e-01,1.012154,4.540470e-02,1.082523,7.670879e-01,1.360812,4.540470e-02,1.454705,9.357184e-01,1.612186,4.540470e-02,1.731502,1.072913e+00,1.832716,4.540470e-02,1.951164,1.195204e+00,1.950666,4.540470e-02,2.042863,1.816188e+00
min,1.000100e+04,1.000000e+00,1.980000e+03,1.000000e+00,-1.435731e+08,1.835761e-01,1.313388e+00,1.000000e+00,1.562500e-02,-9.478570e+00,-9.780850e+00,-1.484859e+01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.116714e+01,-1.684451e+01,-1.874448e+01,-1.948913e+01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.835761e-01,1.313388e+00,1.000000e+00,0.183576,1.313388e+00,1.000000e+00,0.183576,1.313388e+00,1.000000e+00,0.183576,1.313388e+00,1.000000e+00,0.183576,1.313388e+00,1.000000e+00,0.183576,1.313388e+00,1.000000e+00,0.183576,1.313388e+00,1.000000e+00,-9.478570e+00,-10.075663,-3.612986e-01,-8.615118,-9.780850e+00,-10.075663,-3.612986e-01,-9.239290,-1.116714e+01,-10.075663,-3.612986e-01,-